# PyCRAM Presentation

In [1]:
import pycram

Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='base_laser_link']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='wide_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='narrow_stereo_optical_frame']
Unknown attribute "type" in /robot[@name='pr2']/link[@name='laser_tilt_link']


# Bullet World

The BulletWorld is the internal simulation of PyCRAM. You can simulate different actions and reason about the outcome of different actions. 

It is possible to spawn objects and robots into the BulletWorld, these objects can come from URDF, OBJ or STL files. 

A BulletWorld can be created by simply creating an object of the BulletWorld class. 

In [2]:
from pycram.bullet_world import BulletWorld, Object

world = BulletWorld()

The world can also be closed with the 'exit' method

In [3]:
world.exit()

The BulletWorld allows to render images from arbitrary positoins. In the following example we render images with the camera at the position [0.3, 0, 1] and pointing towards [1, 0, 1], so we are looking upwards along the x-axis. 

The renderer returns 3 different kinds of images which are also shown at the left side of the BulletWorld window. These images are:
* A RGB image which shows everything like it is rendered in the BulletWorld window, just from another perspective. 
* A depth image which consists of distance values from the camera towards the objects in the field of view. 
* A segmentation mask image which segments the image into the different objects displayed. The segmentation is done by assigning every pixel the unique id of the object that is displayed there. 

In [3]:
from pycram.bullet_world_reasoning import _get_images_for_target

_get_images_for_target([[1, 0, 1], [0, 0, 0, 1]], [[0.3, 0, 1], [0, 0, 0, 1]])

[array([[[255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         ...,
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255]],
 
        [[255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         ...,
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255]],
 
        [[255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         ...,
         [255, 255, 255, 255],
         [255, 255, 255, 255],
         [255, 255, 255, 255]],
 
        ...,
 
        [[239, 239, 239, 255],
         [239, 239, 239, 255],
         [239, 239, 239, 255],
         ...,
         [239, 239, 239, 255],
         [239, 239, 239, 255],
         [239, 239, 239, 255]],
 
        [[239, 239, 239, 255],
         [239, 239, 239, 255],
         [239, 239, 239, 255],
         ...,
         [239, 239, 239, 255],
         [239, 239, 239, 255],
    

## Objects
Everything that is located inside the BulletWorld is an Object. 
Objects can be created from URDF, OBJ or STL files. Since everything is of type Object a robot might share the same methods as a milk (with some limitations).

Signature:
Object:
* Name 
* Type
* Filename or Filepath

 Optional:
 * Position
 * Orientation
 * World 
 * Color 
 * Ignore Cached Files

If there is only a filename and no path PyCRAM will check in the resource directory if there is a matching file. 


In [4]:
milk = Object("Milk", "milk", "milk.stl")

Objects provide methods to change the position and rotation, change the color, attach other objects, set the state of joints if the objects has any or get the position and orientation of a link. 

These methods are the same for every Object, however since some Objects may not have joints or more than one link methods related to these will not work. 

In [5]:
milk.set_position([1, 0, 0])

To remove an Object from the BulletWorld just call the 'remove' method on the Object.

In [6]:
milk.remove()

Since everything inside the BulletWorld is an Object, even a complex environment Object like the kitchen can be spawned in the same way as the milk.

In [7]:
kitchen = Object("kitchen", "environment", "kitchen.urdf")

## Costmaps

Costmaps are a way to get positions with respect to certain criterias. 
The currently available costmaps are:
* Occupancy Costmap
* Visibility Costmap
* Semantic Costmap 
* Gaussian Costmap

It is also possible to merge multiple costmaps to combine different criteria.

### Visibility Costmaps
Visibility costmaps determine every position, around a target position, from which the target is visible. Visibility Costmaps are able to work with cameras that are moveable in height, for example if the robot has a movable torso. 

In [8]:
import pycram.costmaps as cm
v = cm.VisibilityCostmap(1.27, 1.60, size=300, resolution=0.02, origin=[[0, 0, 0.1], [0, 0, 0, 1]])

In [9]:
v.visualize()

In [10]:
v.close_visualization()

### Occupancy Costmap
Is valid for every position where the robot can be placed without colliding with an object.

In [11]:
o = cm.OccupancyCostmap(0.2, from_ros=False, size=300, resolution=0.02, origin=[[0, 0, 0.1], [0, 0, 0, 1]])

In [ ]:
s = cm.SemanticCostmap(kitchen, "kitchen_island_surface", size=100, resolution=0.02)

g = cm.GaussianCostmap(200, 15, resolution=0.02)

You can visualize the costmap in the BulletWorld to get an impression what information is actually contained in the costmap. With this you could also check if the costmap was created correctly. 
Visualization can be done via the 'visualize' method of each costmap.

In [12]:
o.visualize()

In [13]:
o.close_visualization()

It is also possible to combine two costmap, this will result in a new costmap with the same size which contains the information of both previous costmaps. Combination is done by checking for each position in the two costmaps if they are zero, in this case to same position in the new costmap will also be zero in any other case the new position will be the normalized product of the two combined costmaps.

In [14]:
ov = o + v

In [15]:
ov.visualize()

In [16]:
ov.close_visualization()

## Bullet World Reasoning 
Allows for geometric reasoning in the BulletWorld. At the moment the following types of reasoning are supported:
* Stable
* Contact
* Visible 
* Occluding 
* Reachable 
* Blocking
* Supporting

To show the geometric reasoning we first a robot as well as the milk Object again.

In [17]:
import pycram.bullet_world_reasoning as btr
milk = Object("Milk", "milk", "milk.stl", position=[1, 0, 1])
pr2 = Object("pr2", "robot", "pr2.urdf")

We start with testing for visibility 

In [18]:
milk.set_position([1,0,1])
visible = btr.visible(milk, pr2.get_link_position_and_orientation("wide_stereo_optical_frame"))
print(f"Milk visible: {visible}")

Milk visible: True


In [19]:
milk.set_position([1, 0, 0.05])

plane = BulletWorld.current_bullet_world.objects[0]
contact = btr.contact(milk, plane)
print(f"Milk is in contact with the floor: {contact}")

Milk is in contact with the floor: True


In [20]:
milk.set_position([0.6, -0.5, 0.7])

reachable = btr.reachable(milk, pr2, "r_gripper_tool_frame")
print(f"Milk is reachable for the PR2: {reachable}")

Milk is reachable for the PR2: True


# Designator
Designator are symbolic descriptions of Actions, Motions, Objects or Locations. In PyCRAM the different types of designators are representet by a class which takes a description, the description then tells the designator what to do. 

For example, let's look at a Motion Designator to move the robot to a specific location. 



## Motion Designator

When using a Motion Designator you need to specify which Process Module needs to be used, either the Process Module for the real or the simulated robot. This can be done either with a decorator which can be added to a function and then executes every designator in this function on the specified robot. The other possibility is a "with" scope which wraps a code piece. 

These two ways can also be combined, you could write a function which should be executed on the real robot and in the function is a "with" scope which executes something on the simulated robot for reasoning purposes. 

In [21]:
from pycram.designators.motion_designator import *
from pycram.process_module import simulated_robot, with_simulated_robot

description = MoveMotion(target=[[1, 0, 0], [0, 0, 0, 1]])

with simulated_robot:
    description.resolve().perform()
    


In [22]:
from pycram.process_module import with_simulated_robot

@with_simulated_robot
def move():
    MoveMotion(target=[[0, 0, 0], [0, 0, 0, 1]]).resolve().perform()

move()

Other implemented Motion Designator descriptions are:
* Pick up
* Place
* Accessing
* Move TCP
* Looking
* Move Gripper
* Detecting
* Move Arm Joint 
* World State Detecting 

## Object Designator

Object Designator represent objects, these objects could either be from the BulletWorld or the real world. Object Designator are used, for example, by the PickUpAction to know which object should be picked up.

In [23]:
from pycram.designators.object_designator import *

milk_desig = BelieveObject(names=["Milk"])
milk_desig.resolve()

BelieveObject.Object(name='Milk', type='milk', pose=((0.6, -0.5, 0.7), (0.0, 0.0, 0.0, 1.0)), bullet_world_object=<pycram.bullet_world.Object object at 0x7f8221c59820>)

## Location Designator
Location Designator can create a position in cartisian space from a symbolic desctiption

In [24]:
from pycram.designators.location_designator import *
from pycram.designators.object_designator import *

robot_desig = BelieveObject(types=["robot"]).resolve()
milk_desig = BelieveObject(names=["Milk"]).resolve()
location_desig = CostmapLocation(target=milk_desig, visible_for=robot_desig)

print(f"Resolved: {location_desig.resolve()}")
print()

for pose in location_desig:
    print(pose)
                              

Resolved: CostmapLocation.Location(pose=[[0.7800000309944153, -0.7200000286102295, 0.0], [0.0, 0.0, 0.9036696869551967, 0.42823019146050045]], reachable_arms=None)

CostmapLocation.Location(pose=[[0.7800000309944153, -0.7200000286102295, 0.0], [0.0, 0.0, 0.9036696869551967, 0.42823019146050045]], reachable_arms=None)
CostmapLocation.Location(pose=[[0.48000001907348633, -0.7599999904632568, 0.0], [0.0, 0.0, 0.5389535530902271, 0.8423354840035054]], reachable_arms=None)
CostmapLocation.Location(pose=[[0.7800000309944153, -0.699999988079071, 0.0], [0.0, 0.0, 0.913500086834353, 0.40683853228723244]], reachable_arms=None)
CostmapLocation.Location(pose=[[0.40000003576278687, -0.7200000286102295, 0.0], [0.0, 0.0, 0.4045536551285547, 0.9145142645809995]], reachable_arms=None)
CostmapLocation.Location(pose=[[0.40000003576278687, -0.7599999904632568, 0.0], [0.0, 0.0, 0.44175179320244673, 0.8971373101161397]], reachable_arms=None)
CostmapLocation.Location(pose=[[0.7800000309944153, -0.74000000953

# Action Designator
Action Designator are used to describe high-level actions. Action Designator are usually composed of other Designators to describe the high-level action in detail. 

In [25]:
from pycram.designators.action_designator import *
from pycram.resolver.plans import Arms
from pycram.enums import Arms

with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()


# Making a simple plan
To get familiar with the PyCRAM Framework we will write a simple pick and place plan. This plan will let the robot grab a cereal box from the kitchen counter and place it on the kitchen island. This is a simple pick and place plan.

In [26]:
from pycram.designators.object_designator import *
cereal = Object("cereal", "cereal", "breakfast_cereal.stl", position=[1.4, 1, 0.95])


In [27]:
cereal_desig = ObjectDesignatorDescription(names=["cereal"])
kitchen_desig = ObjectDesignatorDescription(names=["kitchen"])
robot_desig = ObjectDesignatorDescription(names=["pr2"]).resolve()
with simulated_robot:
    ParkArmsAction([Arms.BOTH]).resolve().perform()

    MoveTorsoAction([0.3]).resolve().perform()

    pickup_pose = CostmapLocation(target=cereal_desig.resolve(), reachable_for=robot_desig).resolve()
    pickup_arm = pickup_pose.reachable_arms[0]

    NavigateAction(target_locations=[pickup_pose.pose]).resolve().perform()

    PickUpAction(object_designator_description=cereal_desig, arms=[pickup_arm], grasps=["front"]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()

    place_island = SemanticCostmapLocation("kitchen_island_surface", kitchen_desig.resolve(), cereal_desig.resolve()).resolve()

    place_stand = CostmapLocation(place_island.pose, reachable_for=robot_desig, reachable_arm=pickup_arm).resolve()

    NavigateAction(target_locations=[place_stand.pose]).resolve().perform()

    PlaceAction(cereal_desig, target_locations=[place_island.pose], arms=[pickup_arm]).resolve().perform()

    ParkArmsAction([Arms.BOTH]).resolve().perform()
    
    
    

# Task Trees
Task trees are a hirachical representation of all Actions involved in a plan. The Task tree can later be used to inspect and restructre the execution order of Actions in the plan.

In [29]:
import pycram.task
import anytree
tt = pycram.task.task_tree
print(anytree.RenderTree(tt))

no_operation()
├── perform(ParkArmsAction, )
├── perform(ParkArmsAction, )
├── perform(MoveTorsoAction, )
├── perform(NavigateAction, )
├── perform(PickUpAction, )
├── perform(ParkArmsAction, )
├── perform(NavigateAction, )
├── perform(PlaceAction, )
└── perform(ParkArmsAction, )


In [30]:
from anytree.dotexport import RenderTreeGraph, DotExporter
RenderTreeGraph(tt).to_picture("tree.png")

# ORM


In [31]:
import sqlalchemy
import sqlalchemy.orm
import pycram.orm.base
import pycram.orm.task
import pycram.orm.object_designator
import pycram.orm.motion_designator
import pycram.orm.action_designator

engine = sqlalchemy.create_engine("sqlite+pysqlite:///:memory:", echo=False)
session = sqlalchemy.orm.Session(bind=engine)
pycram.orm.base.Base.metadata.create_all(engine)
session.commit()


tt.insert(session)

pycram.orm.task.TaskTreeNode(1, 1, 2023-04-25 16:33:54.418393, None, RUNNING, None)

In [32]:
navigations = session.query(pycram.orm.action_designator.NavigateAction).all()
print(*navigations, sep="\n")

pycram.orm.action_designator.NavigateAction(4, Navigate, 1, 1)
pycram.orm.action_designator.NavigateAction(7, Navigate, 3, 3)


In [34]:
navigations = session.query(pycram.orm.action_designator.NavigateAction, 
                            pycram.orm.base.Position, 
                            pycram.orm.base.Quaternion).\
                                join(pycram.orm.base.Position).\
                                join(pycram.orm.base.Quaternion).all()
print(*navigations, sep="\n")

(pycram.orm.action_designator.NavigateAction(4, Navigate, 1, 1), pycram.orm.base.Position(1, 0.7799999713897705, 0.7400000095367432, 0.0), pycram.orm.base.Quaternion(1, 0.0, 0.0, 0.1972379359404665, 0.9803556480308279))
(pycram.orm.action_designator.NavigateAction(7, Navigate, 3, 3), pycram.orm.base.Position(3, -1.9075000286102295, 0.7792000770568848, 0.0), pycram.orm.base.Quaternion(3, 0.0, 0.0, 0.16439898301071468, 0.9863939245479175))
